In [1]:
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch("https://localhost:9200", basic_auth=("elastic", "Data_is_Lyfe"), verify_certs=False)

/Users/astrobalaji/anaconda3/envs/semantic_search/lib/python3.8/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [8]:
es.indices.create

In [15]:
es.indices.delete(index = "dummy_index_1")

/tmp/ipykernel_9540/2599647023.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index = "dummy_index_1")


ObjectApiResponse({'acknowledged': True})

In [4]:
settings = {
        "number_of_shards":5
    }

In [7]:
es.indices.create(index="metis_dp_search")

/Users/astrobalaji/anaconda3/envs/semantic_search/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'metis_dp_search'})

In [17]:
es.indices.create(index = "test_index")

/tmp/ipykernel_9540/3571346099.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index = "test_index")


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [19]:
test_doc = {
    "title": "Joby",
    "description": "he's the rockstar!!"
}

In [21]:
es.index(index = "test_index", document = test_doc)

/tmp/ipykernel_9540/2288605334.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index = "test_index", document = test_doc)


ObjectApiResponse({'_index': 'test_index', '_type': '_doc', '_id': 'DQhUPYsBaWvlsqm__w6f', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [22]:
es.search?

In [26]:
qr = {"match_all":{}}

In [30]:
res = es.search(query = qr, index = "test_index")

/tmp/ipykernel_9540/2337092653.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(query = qr, index = "test_index")


In [36]:
res["hits"]["hits"][0]["_source"]

{'title': 'Joby', 'description': "he's the rockstar!!"}

In [43]:
dict(res)["hits"]["hits"][0]["_source"]

{'title': 'Joby', 'description': "he's the rockstar!!"}

In [45]:
es.indices.delete(index = "test_index")

/tmp/ipykernel_9540/1626204862.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index = "test_index")


ObjectApiResponse({'acknowledged': True})

In [46]:
from elasticsearch.helpers import bulk

In [47]:
bulk?

In [48]:
import pandas as pd

In [49]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes")

In [9]:
import json

In [10]:
with open("combined_dataprod.json") as f:
    doc = json.load(f)

In [11]:
doc.keys()

dict_keys(['title', 'description', 'title_vec', 'description_vec'])

In [57]:
es.indices.create(index = "dp_test")

/tmp/ipykernel_9540/1377089035.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index = "dp_test")


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dp_test'})

In [58]:
es.index(index = "dp_test", document = doc)

/tmp/ipykernel_9540/2000744501.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index = "dp_test", document = doc)


ObjectApiResponse({'_index': 'dp_test', '_type': '_doc', '_id': 'DghePYsBaWvlsqm_GQ6v', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [61]:
dict(es.indices.get_mapping(index = "dp_test"))["dp_test"]

/tmp/ipykernel_9540/1771099122.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  dict(es.indices.get_mapping(index = "dp_test"))["dp_test"]


{'mappings': {'properties': {'description': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'description_vec': {'type': 'float'},
   'title': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'title_vec': {'type': 'float'}}}}

In [ ]:
mapping = {
    "mappings":{
        "properties":{
            "title": {"type":"text"},
            "description":{"type":"text"}
        }
    }
}

In [63]:
len(doc["title_vec"])

384

In [17]:
mapping_out = {'properties': {'description': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'description_vec': {'type': 'dense_vector', 'dims':384},
   'title': {'type': 'text',
    'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
   'title_vec': {'type': 'dense_vector', 'dims':384}}}

In [15]:
mapping_out["properties"]["title_vec"]

{'type': 'dense_vec', 'dims': 384}

In [16]:
mapping_out["properties"]["description_vec"]

{'type': 'dense_vec', 'dims': 384}

In [18]:
es.indices.create(index = "dp_test_1", mappings=mapping_out)

/Users/astrobalaji/anaconda3/envs/semantic_search/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dp_test_1'})

In [19]:
es.index(index = "dp_test_1", document=doc)

/Users/astrobalaji/anaconda3/envs/semantic_search/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'_index': 'dp_test_1', '_id': 'iV_gSIsBlKtNqqT2E-K_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [36]:
test_phrase = "ecommerce"

In [37]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [38]:
query_vec = model.encode(test_phrase)

In [39]:
len(query_vec)

384

In [42]:
qr = {
"script_score": {
"query": {"match_all": {}},
"script": {
"source": "cosineSimilarity(params.query_vector, 'title_vec') + 1.0",
"params": {"query_vector": query_vec}
}
}
}

In [43]:
es.search(query=qr, index = "dp_test_1")["hits"]

/Users/astrobalaji/anaconda3/envs/semantic_search/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'total': {'value': 1, 'relation': 'eq'},
 'max_score': 1.3488992,
 'hits': [{'_index': 'dp_test_1',
   '_id': 'iV_gSIsBlKtNqqT2E-K_',
   '_score': 1.3488992,
   '_source': {'title': 'Customer_360',
    'description': 'Customer data from various sources, enabling a holistic view of customers. ',
    'title_vec': [-0.0750977098941803,
     -0.032000958919525146,
     -0.03437187150120735,
     -0.054321449249982834,
     0.009169877506792545,
     -0.04182756319642067,
     0.10451316833496094,
     0.009180744178593159,
     0.017489438876509666,
     -0.06019215285778046,
     0.07124654948711395,
     -0.020160837098956108,
     0.034009113907814026,
     -0.02023700438439846,
     0.03482156991958618,
     -0.02517235279083252,
     -0.0056594256311655045,
     0.02753726951777935,
     -0.10023246705532074,
     0.006730086635798216,
     -0.1408040076494217,
     0.0020834109745919704,
     -0.06547527760267258,
     0.05336359515786171,
     -0.05474458634853363,
     0.041959114